Load points from SLAM capture.

In [1]:
import msgpack

file_path = "./2020-05-15_demo.msg"

with open(file_path, "rb") as data_file:
    byte_data = data_file.read()

data_loaded = msgpack.unpackb(byte_data)

Extract points from msqpack format to 3D point array.

Also invert the x and y axis (because apparently everything is upside down from OpenVSLAM to Open3D ¯\\\_(ツ)_/¯ ).

In [58]:
import numpy as np

pts_new = np.array([item[b'pos_w'] for item in data_loaded[b'landmarks'].values()])

R = np.array([[1,0,0],
              [0,0,-1],
              [0,1,0]])
pts_new = np.dot(pts_new, R.T)

In [72]:
import open3d as o3d

def remove_outliers(cloud):
    voxel_down_pcd = cloud.voxel_down_sample(voxel_size=0.02)
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.0)
    return voxel_down_pcd.select_down_sample(ind)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(
    pts_new
)

inlier_cloud = remove_outliers(pcd)

Visualize the pointcloud with the ability to have black backgounrd (by pressing k).

In [73]:
def change_background_to_black(vis):
    opt = vis.get_render_option()
    opt.background_color = 1 - opt.background_color
    return False

key_to_callback = {
    ord("K"): change_background_to_black
}

# Draw everything
o3d.visualization.draw_geometries_with_key_callbacks([inlier_cloud], key_to_callback)